In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

!pip install keras==2.2.3 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#importing libraires
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.activations import relu,softmax

from tensorflow.python.keras.applications import ResNet50
from keras.applications import VGG16


#Image Augmentation libraries
from skimage.util import random_noise
from skimage.util import invert
from skimage.transform import rotate
from skimage.exposure import rescale_intensity
from skimage.exposure import adjust_gamma
from skimage.exposure import adjust_log
from skimage.exposure import adjust_sigmoid
from scipy.ndimage import uniform_filter

## Lets see what all we got as input

### Image to ID Mapping

In [ ]:
train_df = pd.read_csv('../input/train.csv')
print('The shape of train df is ',train_df.shape)
train_df.head(5)

In [ ]:
print('Total unique kinds of images are ', train_df['Image'].nunique())
print('Total unique kinds of ids are ', train_df['Id'].nunique())
ids_count = train_df.groupby('Id').count().reset_index().sort_values(['Image'],ascending = False)
ids_count.head(5)

In [ ]:
#Training set would be only those images which has the whale ID with them
train_df_with_ids = train_df[(train_df['Id'] != 'new_whale')]
train_df_with_ids = train_df_with_ids.reset_index()[['Image','Id']]

In [ ]:
train_df_with_ids.head(5)

In [ ]:
train_df_with_ids.shape


In [ ]:
#Augmentation method
aug_func_count = 2

#random noise
def random_noise_func(img):
    new_img = random_noise(img)
    return new_img

#inversion
def invert_func(img):
    new_img = invert(img)
    return new_img

#rotate
def rotate_func(img):
    new_img = rotate(img,15)
    return new_img


#change contrast
def contrast_func(img):
    v_min,v_max = np.percentile(img,(0.2,99.8))  
    new_img = rescale_intensity(img,in_range = (v_min,v_max))
    return new_img


#gamma correction
def gamma_func(img):
    new_img = adjust_gamma(img,gamma = 0.4,gain=0.9)
    return new_img


#log correction
def log_func(img):
    new_img = adjust_log(img)
    return new_img


#Sigmoid Correction
def sigmoid_func(img):
    new_img = adjust_sigmoid(img)
    return new_img

#Horizontal Flip
def H_flip_func(img):
    new_img = img[:,::-1]
    return new_img


#Blur image
def Blur_func(img):
    new_img = uniform_filter(img,size=(8,8,1))
    return new_img


In [ ]:
image_count_cutoff = 3
ids_need_aug = ids_count[ids_count['Image'] <image_count_cutoff]['Id']

imgs_need_aug = train_df_with_ids[train_df_with_ids['Id'].isin(ids_need_aug)][['Image','Id']].iloc[0:10]


In [ ]:
ids_need_aug.count()

In [ ]:
extra_images_count = len(imgs_need_aug) * aug_func_count
extra_images_count

;

##### Based on above distribution, we have lots pf IDs with very less images. Around 2000 IDs have only 1 image (Very High imbalanced Dataset)

### Lets work on this imbalanced data


### First option - Put Ids with less images as new_whale
Identify all the whale_ids with images less than 4 or 5 (Decide the cutoffs) We can take less than 4
Replace thier id with new whale - This way we should have less number of target variables
We do not want to train my model for new whale-- Ques - Do softmax function has any not condition
Train the model . Output shud be biased towards new_whale in that way-

### Second option - Data augmentation for less images

### Images in train dataset

In [ ]:
#test_no = train_df_with_ids.count()[0]  # Choose Test_no as a perfect square number like 16,25,36 because it will help in displaying image in the grid view later
test_no = 25
df = train_df_with_ids.iloc[:test_no]

In [ ]:

# # '../input/train/'  contains all the images 25361 images
# count_of_images = df.shape[0]
# target_size = (200,200,3)

# #Our final X_train matrix would be shape of Count_of_images,shape of single image
# #We would take 100,100,3 as a sample image size and interpolate every image to this size
# X = np.zeros((count_of_images,target_size[0],target_size[1],target_size[2]))

# #Now we shall create a function to read every image and convert it into array and do preprocessing

# def prepare_image(each_image):
#     img = plt.imread('../input/train/' + each_image)  # Reading each image
#     img = resize(img,target_size,anti_aliasing=True)
#     #img_arr = image.img_to_array(img)                                               #converting it to an array 
#     #img_arr = preprocess_input(img_arr)                                             #Preprocessing input
#     return img

# for i in range(0,count_of_images):
#     each_img_array = prepare_image(df['Image'][i])
#     X[i] = each_img_array
    
#     if i%1000 == 0:
#         print('processing image',i+1)
    
   
# print(X.shape)




# '../input/train/'  contains all the images 25361 images
count_of_images = df.shape[0] + extra_images_count
target_size = (100,100,3)

#Our final X_train matrix would be shape of Count_of_images,shape of single image
#We would take 100,100,3 as a sample image size and interpolate every image to this size
X = np.zeros((count_of_images,target_size[0],target_size[1],target_size[2]))
y = []
#Now we shall create a function to read every image and convert it into array and do preprocessing

def prepare_image(each_image):
    img = plt.imread('../input/train/' + each_image)  # Reading each image
    img = resize(img,target_size,anti_aliasing=True)
    return img

i= 0
while i < count_of_images:
    if i < df.shape[0]: 
        each_img_array = prepare_image(df['Image'][i])
        X[i] = each_img_array
        y.append(df['Id'][i])
        i = i+1
    else:
        j = 0
        each_img_array = prepare_image(imgs_need_aug['Image'].iloc[j])
        
        #random noise
        new_img_array = random_noise_func(each_img_array)
        X[i] = new_img_array
        y.append(imgs_need_aug['Id'].iloc[j])
        
        #invert
        new_img_array = random_noise_func(each_img_array)
        X[i+1] = new_img_array
        y.append(imgs_need_aug['Id'].iloc[j])
        
#         #rotate
#         new_img_array = rotate_func(each_img_array)
#         X[i+2] = new_img_array
#         y.append(imgs_need_aug['Id'].iloc[j])
        
#         #contrast
#         new_img_array = contrast_func(each_img_array)
#         X[i+3] = new_img_array
#         y.append(imgs_need_aug['Id'].iloc[j])
        
#         #gamma
#         new_img_array = gamma_func(each_img_array)
#         X[i+4] = new_img_array
#         y.append(imgs_need_aug['Id'].iloc[j])
        
#         #log
#         new_img_array = log_func(each_img_array)
#         X[i+5] = new_img_array
#         y.append(imgs_need_aug['Id'].iloc[j])
        
        
#         #sigmoid
#         new_img_array = sigmoid_func(each_img_array)
#         X[i+6] = new_img_array
#         y.append(imgs_need_aug['Id'].iloc[j])
        
        
#         #horizontal
#         new_img_array = H_flip_func(each_img_array)
#         X[i+7] = new_img_array
#         y.append(imgs_need_aug['Id'].iloc[j])
        
        
#         #Blur
#         new_img_array = Blur_func(each_img_array)
#         X[i+8] = new_img_array
#         y.append(imgs_need_aug['Id'].iloc[j])
        

        i = i + aug_func_count
        j= j+1
        
    if i%1000 == 0:
        print('processing image',i+1)
    
   
print(X.shape)
print(len(y))


In [ ]:
#preparing labels - There are total 5005 labels - we shall use label encoders on this
#And then go ahead with one hot encoding
le = LabelEncoder()
ohe = OneHotEncoder(categories = 'auto')

y1 = pd.DataFrame(y)
#y = np.array(df['Id'])
y = np.array(y1)
y = le.fit_transform(y)
y = y.reshape(len(y),1)
y = ohe.fit_transform(y)



In [ ]:
y.shape

In [ ]:
num_classes = y.shape[1]
print(num_classes)

#### Preprocessing images

In [ ]:
#X.shape

In [ ]:
#X_mean
# X_mean = np.mean(X,axis =0)
# plt.imshow(X_mean)

In [ ]:
#X_variance
# X_std = np.std(X,axis=0)
# plt.imshow(X_std)

#### Grid View

In [ ]:
# grid_dim = np.int(np.sqrt(test_no))


# fig = plt.figure(figsize=(20,20))
# for i in range(test_no):
#     temp = fig.add_subplot(grid_dim,grid_dim,i+1)
#     temp.set_xticks([])
#     temp.set_yticks([])
#     plt.imshow(X[i])

# fig.show()  

### Model Creation

In [ ]:
#resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
y.shape

In [ ]:
# model = Sequential()
# model.add(Conv2D(filters = 32,kernel_size = (7,7),strides=2,activation= 'relu', padding='valid',name = 'conv0', input_shape = (100, 100, 3)))
# model.add(Conv2D(16, kernel_size=(3,3),strides=2, activation= 'relu'))
# model.add(Conv2D(16, kernel_size=(3,3), activation= 'relu'))
# model.add(Flatten())
# model.add(Dense(5005, activation= 'softmax'))

# model.summary()



# model = Sequential()

# model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

# model.add(BatchNormalization(axis = 3, name = 'bn0'))
# model.add(Activation('relu'))

# model.add(MaxPooling2D((2, 2), name='max_pool'))
# model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
# model.add(Activation('relu'))
# model.add(AveragePooling2D((3, 3), name='avg_pool'))

# model.add(Flatten())
# model.add(Dense(500, activation="relu", name='rl'))
# model.add(Dropout(0.8))
# model.add(Dense(num_classes, activation='softmax', name='sm'))

# model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# model.summary()


#Transfer learning
resnet_weights_path = 'imagenet'

model = Sequential()
#model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
model.add(VGG16(include_top=False, pooling='avg', weights=resnet_weights_path))
#model.add(ResNet50( pooling='avg', weights=resnet_weights_path))
#model.add(ResNet50(include_top=True, weights='imagenet', input_tensor=None, pooling=None))

model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
model.layers[0].trainable = False

model.summary()




In [ ]:
#fitting model
model.compile(optimizer= 'adam',loss='categorical_crossentropy',metrics= ['accuracy'])
model.fit(X,y,validation_split=0.2, epochs=3,batch_size = 250)


In [ ]:
#Predictions
# count_of_images = df.shape[0] + extra_images_count
# target_size = (100,100,3)

# #Our final X_train matrix would be shape of Count_of_images,shape of single image
# #We would take 100,100,3 as a sample image size and interpolate every image to this size
# X = np.zeros((count_of_images,target_size[0],target_size[1],target_size[2]))
# y = []
#Now we shall create a function to read every image and convert it into array and do preprocessing

# def prepare_image(each_image):
#     img = plt.imread('../input/test/' + each_image)  # Reading each image
#     img = resize(img,target_size,anti_aliasing=True)
#     return img


#predictions = model.predict(np.random.rand(2,100,100,3))

test.shape

In [ ]:
predictions.argsort()

In [ ]:
sample_image = image.load_img('../input/train/0000e88ab.jpg')

In [ ]:
sample_image.size

In [ ]:
sample_image

In [ ]:
for fig in train_df['Image'][:10]:
    img = image.load_img('../input/train/' + fig )
    print(img.size)

In [ ]:
# so we have images avaiabe in different sizes
#lets make them all in smaller size(less pizels,100, 100, 3)
#image.load_img('../input/train/0000e88ab.jpg',target_size=(100,100))
image_name = '8f9f4e8c2.jpg'
#image.load_img('../input/train/' + image_name).shape
temp = plt.imread(fname = '../input/train/' + image_name, format = 'jpg')

In [ ]:
plt.imshow(temp)
# temp2 = np.zeros((1050,1310,3))
# for i in range(3):
#     temp2[:,:,i] = temp[:,:,i].T
# temp2.shape
# plt.imshow(temp2)

In [ ]:
sample_image_chgd =image.load_img('../input/train/' + image_name)

In [ ]:
#image.img_to_array(sample_image_chgd)
sample_image_chgd.size

In [ ]:
x = image.img_to_array(sample_image_chgd)

In [ ]:
x.shape

In [ ]:
x[:,:,1]

In [ ]:
y= np.random.randint(low = 1,high= 10,size=(1,4,3))
#plt.imshow(y)
y

## x = preprocess_input(x)


In [ ]:
x

In [ ]:
X_train = np.zeros((3, 2, 2, 3))

In [ ]:
X_train[1] = x

In [ ]:
X_train

In [ ]:
from scipy import misc
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
np.ones((2,2,1))

In [ ]:
ascent = misc.ascent()
#pic = np.ones((5,5)*30
# pic = np.random.randint(0,3,5*5).reshape(5,5)
# pic

In [ ]:
ascent.s

In [ ]:
plt.imshow(ascent)
#plt.imshow(pic,cmap = 'gray')

### Junk Code- Ignore this section

In [ ]:
#a loop to view multiple images of same id
'''
image_list = [
'0829ae94e.jpg','10b694367.jpg','180056116.jpg','18f817e48.jpg','2354879ce.jpg','294ab237b.jpg','3d7f4c7d5.jpg','45adc4f95.jpg','4cc7c734b.jpg','4e6c0e6da.jpg','4fa94d7be.jpg','51072b758.jpg','55439f4de.jpg','557c56974.jpg','64fc5e880.jpg','73135d38c.jpg','75ad5e6b0.jpg','82b65b762.jpg'
]

for i in range(len(image_list)):
    image = plt.imread('../input/train/' + image_list[i])
    print(image.shape[1])
    plt.imshow(image)
    plt.figure()
#plt.imshow(image)
'''

#appending Resolution of the image with the image ID
def get_resolution(img):
    image = plt.imread('../input/train/' + img)
    #return image.shape[0],image.shape[1],image.shape[1]
    return image.shape

# train_df_1['c'] = train_df['Image'][0:5].apply(lambda x : get_resolution(x))
# #train_df_1['a'],train_df_1['b'],train_df_1['c'] = train_df_1['Image'].apply(lambda x : x[0],x[1],x[2])
# train_df_1['d'] = train_df_1['Image'].apply(lambda x : x[0])
#train_df['Image'][0:5].apply(lambda x : get_resolution(x)))
 
#train_df_1['Res_1'][0:5],train_df_1['Res_2'][0:5],train_df_1['clr_band'][0:5] = train_df['Image'][0:5].apply(lambda x : get_resolution(x))
'''   
train_df['Resolution'] = train_df['Image'].apply(lambda x : get_resolution(x))
train_df['Res_1'] = train_df['Resolution'].apply(lambda x : x[0])
train_df['Res_2'] = train_df['Resolution'].apply(lambda x : x[1])
train_df['Clr_Band'] = train_df['Resolution'].apply(lambda x : x[2])

'''

#saving this csv
#train_df.to_csv('Updated_train_1.csv',index = False)


# for i in range(test_no):
#     #plt.figure(figsize=(5, 1)).add_subplot(nrows = 1,ncols = 5,index=i+1)
#     plt.subplot(1,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.imshow(X_array[i])
    
# plt.show()
  
    
# def prepare_image(each_image):
#     img = image.load_img('../input/train/' + each_image,target_size = target_size)  # Reading each image
#     img_arr = image.img_to_array(img)                                               #converting it to an array 
#     img_arr = preprocess_input(img_arr)                                             #Preprocessing input
#     return img_arr


#dividing data into Train and test
#X_train.shape
#y_train.shape
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

grid_dim = np.int(np.sqrt(test_no))

# fig = plt.figure(figsize=(20,20))
# for i in range(test_no):
#     temp = fig.add_subplot(grid_dim,grid_dim,i+1)
#     temp.set_xticks([])
#     temp.set_yticks([])
#     plt.imshow(X[i])
# fig.show()  

#Dimensionality Reductiion
'''
def rgb_to_gray(img):
    return np.dot(img,[0.299, 0.587, 0.114])

X_bw = np.zeros((X.shape[0],X.shape[1],X.shape[2]))
                
for i in range(X.shape[0]):
    X_bw[i] = rgb_to_gray(X[i])
X_bw.shape
'''

#Normalization - Subtracticting mean and dividing by std
'''
X = X- X_mean
X = X/X_std
'''


#Data Augmentation

#random noise
# from skimage.util import random_noise
# img = X[0]    
# new_img = random_noise(img)
# plt.imshow(img) 

#inversion
# from skimage.util import invert
# img = X[0]    
# new_img = invert(img)
# plt.imshow(img) 

#rotate
# from skimage.transform import rotate
# img = X[0]    
# new_img = rotate(img,30)
# plt.imshow(img) 


#change contrast
# from skimage.exposure import rescale_intensity
# v_min,v_max = np.percentile(img,(0.2,99.8))
# new_img = rescale_intensity(img,in_range = (v_min,v_max))
# plt.imshow(img)

#gamma correction
# from skimage.exposure import adjust_gamma
# new_img = adjust_gamma(img,gamma = 0.4,gain=0.9)
# plt.imshow(img)


#log correction
# from skimage.exposure import adjust_log
# new_img = adjust_log(img)
# plt.imshow(img)

#Sigmoid Correction
# from skimage.exposure import adjust_sigmoid
# new_img = adjust_sigmoid(img)
# plt.imshow(img)


#Horizontal Flip
# new_img = img[:,::-1]
# plt.imshow(img)


#Vertical flip
# new_img = img[::-1,:]
# plt.imshow(img)

#Blur image
# from scipy.ndimage import uniform_filter
# new_img = uniform_filter(img,size=(8,8,1))
# plt.imshow(img)


    
    
   
